# Shopify Subdomain Takeover Scanner

**Dataset-based workflow:**
1. Load CSV dataset (`/kaggle/input/all-leads-merged/results.csv`)
2. Sort by Est Monthly Page Views (highest traffic first)
3. Deep scan with auto-resume support

**Features:**
- ✅ Auto-resume: Progress saved every 50 domains
- ✅ Results backup: Previous results preserved on pull/clone
- ✅ Persistent storage: Progress and results survive notebook restarts

**Files stored in `/kaggle/working/` (persist between sessions):**
- `scan_progress.json` - Tracks last scanned row
- `all_results.json` - Accumulated scan results

## Cell 1: Backup Previous Results (Run Before Clone)

**This cell backs up your scan results before cloning overwrites them.**

Results are saved to `/kaggle/working/all_results.json` which persists outside the repo folder.

In [ ]:
import json
import shutil
import os
from pathlib import Path

# Paths
REPO_RESULTS = '/kaggle/working/subdomain-playground/data/scans/shopify_takeover_candidates.json'
BACKUP_RESULTS = '/kaggle/working/all_results.json'

print("=" * 80)
print("BACKING UP RESULTS")
print("=" * 80)
print("")

# Check if there are results to backup
if os.path.exists(REPO_RESULTS):
    # Load current results from repo
    with open(REPO_RESULTS, 'r') as f:
        current_results = json.load(f)
    
    print(f"Found {len(current_results)} results in repo folder")
    
    # Load existing backup if any
    existing_backup = []
    if os.path.exists(BACKUP_RESULTS):
        with open(BACKUP_RESULTS, 'r') as f:
            existing_backup = json.load(f)
        print(f"Found {len(existing_backup)} results in backup")
    
    # Merge (avoid duplicates by subdomain)
    existing_subdomains = {r['subdomain'] for r in existing_backup}
    new_results = [r for r in current_results if r['subdomain'] not in existing_subdomains]
    
    merged = existing_backup + new_results
    
    # Save merged backup
    with open(BACKUP_RESULTS, 'w') as f:
        json.dump(merged, f, indent=2)
    
    print(f"Added {len(new_results)} new results")
    print(f"✓ Total backed up: {len(merged)} results")
    print(f"✓ Saved to: {BACKUP_RESULTS}")
else:
    if os.path.exists(BACKUP_RESULTS):
        with open(BACKUP_RESULTS, 'r') as f:
            existing = json.load(f)
        print(f"No new results in repo folder")
        print(f"✓ Existing backup preserved: {len(existing)} results")
    else:
        print("No results to backup yet (first run)")

print("")
print("=" * 80)
print("Safe to clone/pull now - results are backed up!")
print("=" * 80)

## Cell 2: Clone Project from GitHub

In [ ]:
%%bash
WORKDIR="/kaggle/working"
PROJECT_DIR="$WORKDIR/subdomain-playground"

echo "=========================================="
echo "Cloning Project from GitHub"
echo "=========================================="

mkdir -p "$WORKDIR"
cd "$WORKDIR"

if [ -d "$PROJECT_DIR" ]; then
    echo "Removing existing copy..."
    rm -rf "$PROJECT_DIR"
fi

git clone --depth 1 https://github.com/sayihhamza/subdomain-playground.git subdomain-playground

if [ ! -d "$PROJECT_DIR" ]; then
    echo "✗ Clone failed!"
    exit 1
fi

cd "$PROJECT_DIR"

# Create data/scans directory for results
mkdir -p data/scans

echo ""
echo "✓ Project cloned successfully!"
echo ""
echo "Project structure:"
ls -lh | head -20

## Cell 3: Install Go 1.24

In [ ]:
%%bash
cd /tmp

if command -v sudo >/dev/null 2>&1; then
    SUDO="sudo"
else
    SUDO=""
fi

echo "=========================================="
echo "Installing Go 1.24.1"
echo "=========================================="

$SUDO rm -rf /usr/local/go 2>/dev/null || true

echo "Downloading Go 1.24.1..."
for i in {1..3}; do
    wget -q https://go.dev/dl/go1.24.1.linux-amd64.tar.gz -O /tmp/go.tar.gz && break || sleep 5
done

if [ ! -f /tmp/go.tar.gz ]; then
    echo "✗ Failed to download Go"
    exit 1
fi

$SUDO tar -C /usr/local -xzf /tmp/go.tar.gz
rm -f /tmp/go.tar.gz

echo ""
echo "✓ Go 1.24.1 installed!"
/usr/local/go/bin/go version

## Cell 4: Build Security Tools

Builds 5 tools (5-8 minutes):
- **subfinder**: Passive subdomain enumeration
- **findomain**: Additional passive enumeration (optional)
- **httpx**: HTTP validation
- **dnsx**: DNS + CNAME validation
- **subzy**: Takeover detection

In [ ]:
%%bash
export PATH=/usr/local/go/bin:$PATH
cd /kaggle/working/subdomain-playground

echo "=========================================="
echo "Building Security Tools"
echo "=========================================="
echo ""

mkdir -p bin

# Build Go tools
build_tool() {
    local name=$1
    local repo=$2
    local index=$3
    local total=$4
    local max_attempts=3
    
    echo "[$index/$total] Building $name..."
    
    for attempt in $(seq 1 $max_attempts); do
        if [ $attempt -gt 1 ]; then
            echo "  Retry $attempt/$max_attempts..."
            sleep 2
        fi
        
        if GOBIN=$(pwd)/bin /usr/local/go/bin/go install -v ${repo}@latest 2>&1; then
            if [ -f "bin/$name" ]; then
                echo "  ✓ $name built successfully"
                return 0
            fi
        fi
    done
    
    echo "  ✗ Failed to build $name"
    return 1
}

# Download precompiled binary
download_binary() {
    local name=$1
    local url=$2
    local index=$3
    local total=$4
    local max_attempts=3
    
    echo "[$index/$total] Downloading $name..."
    
    for attempt in $(seq 1 $max_attempts); do
        if [ $attempt -gt 1 ]; then
            echo "  Retry $attempt/$max_attempts..."
            sleep 2
        fi
        
        if curl -sL "$url" -o /tmp/${name}.zip 2>&1; then
            if unzip -q /tmp/${name}.zip -d bin/ 2>&1; then
                if [ -f "bin/$name" ]; then
                    chmod +x "bin/$name"
                    echo "  ✓ $name downloaded successfully"
                    rm -f /tmp/${name}.zip
                    return 0
                fi
            fi
        fi
    done
    
    echo "  ⚠️  $name download failed (optional)"
    rm -f /tmp/${name}.zip
    return 1
}

# Build required tools
build_tool "subfinder" "github.com/projectdiscovery/subfinder/v2/cmd/subfinder" "1" "5"
echo ""

download_binary "findomain" "https://github.com/Findomain/Findomain/releases/latest/download/findomain-linux-i386.zip" "2" "5"
echo ""

build_tool "httpx" "github.com/projectdiscovery/httpx/cmd/httpx" "3" "5"
echo ""

build_tool "dnsx" "github.com/projectdiscovery/dnsx/cmd/dnsx" "4" "5"
echo ""

build_tool "subzy" "github.com/PentestPad/subzy" "5" "5"
echo ""

echo "=========================================="
echo "Verification"
echo "=========================================="

REQUIRED="subfinder httpx dnsx subzy"
TOOLS_OK=true

for tool in $REQUIRED; do
    if [ -f "bin/$tool" ]; then
        echo "  ✓ bin/$tool"
    else
        echo "  ✗ bin/$tool MISSING"
        TOOLS_OK=false
    fi
done

[ -f "bin/findomain" ] && echo "  ✓ bin/findomain (bonus!)" || echo "  ⚠️  bin/findomain (optional)"

if [ "$TOOLS_OK" = false ]; then
    echo ""
    echo "⚠️  Required tools failed - re-run this cell"
    exit 1
fi

echo ""
echo "✓ All required tools built!"

## Cell 5: Set Environment Variables

In [ ]:
import os

os.environ['SUBFINDER_PATH'] = '/kaggle/working/subdomain-playground/bin/subfinder'
os.environ['FINDOMAIN_PATH'] = '/kaggle/working/subdomain-playground/bin/findomain'
os.environ['DNSX_PATH'] = '/kaggle/working/subdomain-playground/bin/dnsx'
os.environ['HTTPX_PATH'] = '/kaggle/working/subdomain-playground/bin/httpx'
os.environ['SUBZY_PATH'] = '/kaggle/working/subdomain-playground/bin/subzy'

os.chdir('/kaggle/working/subdomain-playground')

print("✓ Environment configured")
print(f"✓ Working directory: {os.getcwd()}")

## Cell 6: Load and Sort Dataset

**Configuration:**
- `SORT_BY`: Column to sort by (default: 'Est Monthly Page Views')

**What this cell does:**
1. Loads the full CSV dataset from Kaggle input
2. Sorts by specified column (descending)
3. Creates `df_base_filtered` for Cell 7 and Cell 8

In [ ]:
import pandas as pd

# ========================================
# CONFIGURATION
# ========================================
SORT_BY = 'Est Monthly Page Views'  # Column to sort by

print("=" * 80)
print("LOADING AND SORTING DATASET")
print("=" * 80)
print("")

# Load CSV from Kaggle input
csv_path = '/kaggle/input/all-leads-merged/results.csv'
print(f"Loading: {csv_path}")
df = pd.read_csv(csv_path, low_memory=False)
print(f"✓ Loaded {len(df):,} total rows")
print("")

# Sort by specified column (NO FILTERING)
print(f"Sorting by: {SORT_BY} (descending)")
if SORT_BY in df.columns:
    # Convert to numeric if it's a numeric column
    df['_sort_numeric'] = pd.to_numeric(
        df[SORT_BY].astype(str).str.replace(r'[^\d.]', '', regex=True),
        errors='coerce'
    ).fillna(0)
    df_sorted = df.sort_values('_sort_numeric', ascending=False)
    print(f"✓ Sorted by {SORT_BY}")
else:
    print(f"⚠️  Column '{SORT_BY}' not found - using unsorted")
    df_sorted = df
print("")

# Store base dataset for Cell 7 (preview) and Cell 8 (scan)
df_base_filtered = df_sorted.copy()

print("=" * 80)
print("✓ Dataset loaded and sorted")
print(f"Total available: {len(df_base_filtered):,} rows")
print("")
print("Next steps:")
print("  • Cell 7: Apply filters + preview range")
print("  • Cell 8: Apply filters + scan range")
print("=" * 80)

## Cell 7: Preview Dataset (Optional)

**Configure preview range:**
- `PREVIEW_START_ROW` and `PREVIEW_END_ROW`

**Filters applied (for preview only):**
1. Is_Shopify == 'Yes'
2. Exclude *.myshopify.com domains
3. Exclude HTTP 200 and 429

In [ ]:
import pandas as pd

# ========================================
# PREVIEW CONFIGURATION (Cell 7 only)
# ========================================
PREVIEW_START_ROW = 1       # First row to preview (1-indexed)
PREVIEW_END_ROW = 100       # Last row to preview

print("=" * 120)
print(f"PREVIEW: Rows {PREVIEW_START_ROW} to {PREVIEW_END_ROW} from Sorted Dataset")
print("=" * 120)
print("")
print(f"Source: df_base_filtered from Cell 6 (sorted by {SORT_BY})")
print(f"Total available: {len(df_base_filtered):,} rows")
print("")

# Select preview range from base dataset (sorted, but unfiltered)
df_preview = df_base_filtered.iloc[PREVIEW_START_ROW-1:PREVIEW_END_ROW].copy()
print(f"Selected range: {len(df_preview)} rows")
print("")

# Apply preview filters (for display only)
print("Applying filters for preview...")

# Filter A: Shopify stores only
is_shopify = df_preview['Is_Shopify'] == 'Yes'
print(f"  After Shopify filter: {is_shopify.sum()} rows")

# Filter B: Exclude raw 'myshopify.com' domains
is_custom_domain = ~df_preview['Subdomain'].str.contains('myshopify.com', case=False, na=False)
print(f"  After myshopify.com exclusion: {(is_shopify & is_custom_domain).sum()} rows")

# Filter C: Exclude HTTP 200 and 429
excluded_statuses = [200, 429, '200', '429', 200.0, 429.0]
is_interesting_status = ~df_preview['HTTP_Status'].isin(excluded_statuses)
print(f"  After status exclusion: {(is_shopify & is_custom_domain & is_interesting_status).sum()} rows")

# Apply all filters
df_preview_filtered = df_preview[is_shopify & is_custom_domain & is_interesting_status].copy()
print("")

# Prepare display with original row numbers
display_df = df_preview_filtered.copy()
# Calculate actual row numbers from base dataset
actual_rows = []
for idx in display_df.index:
    actual_row = df_base_filtered.index.get_loc(idx) + 1
    actual_rows.append(actual_row)
display_df.insert(0, 'Row', actual_rows)

# Select columns to display
display_columns = ['Row', 'Subdomain', 'HTTP_Status', SORT_BY, 'CNAME_Record']
display_columns = [col for col in display_columns if col in display_df.columns]

print("=" * 120)
print(f"Displaying {len(display_df)} domains after filters:")
print("=" * 120)
print("")

# Configure pandas display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', 120)

# Display
print(display_df[display_columns].to_string(index=False))

print("")
print("=" * 120)
print("📋 Preview Summary:")
print(f"  • Preview range: Rows {PREVIEW_START_ROW} to {PREVIEW_END_ROW}")
print(f"  • After filters: {len(display_df)} domains")
print(f"  • Sorted by: {SORT_BY} (highest first)")
print(f"  • Filters: Shopify + Custom domain + Exclude status 200/429")
print("")
print("💡 NOTE: Cell 8 scans ALL domains in range (scan.py applies its own filters)")
print("=" * 120)

## Cell 8: Deep Scan with Auto-Resume

**How it works:**
1. Progress saved to `/kaggle/working/scan_progress.json` every 50 domains
2. Results saved to `/kaggle/working/all_results.json` after each session
3. On re-run, automatically resumes from last completed row

**Configuration:**
- `SCAN_END_ROW` - Last row to scan (default: all rows)
- `FORCE_RESTART = True` - Start from row 1, ignoring saved progress

**Manual Resume (if needed):**
```python
import json
with open('/kaggle/working/scan_progress.json', 'w') as f:
    json.dump({'last_row': 2880, 'total_scanned': 2880}, f)
```

In [ ]:
import subprocess
import sys
import pandas as pd
import os
import json
import re
from pathlib import Path
from datetime import datetime

# ========================================
# SCAN CONFIGURATION
# ========================================
SCAN_END_ROW = 1786654       # Last row to scan
FORCE_RESTART = False        # Set to True to start from row 1

# ========================================
# FILE PATHS (persist in /kaggle/working)
# ========================================
PROGRESS_FILE = '/kaggle/working/scan_progress.json'
BACKUP_RESULTS = '/kaggle/working/all_results.json'
REPO_RESULTS = '/kaggle/working/subdomain-playground/data/scans/shopify_takeover_candidates.json'

def load_progress():
    """Load last scanned row from progress file"""
    if os.path.exists(PROGRESS_FILE):
        try:
            with open(PROGRESS_FILE, 'r') as f:
                return json.load(f)
        except:
            pass
    return {'last_row': 0, 'total_scanned': 0}

def save_progress(last_row, total_scanned):
    """Save progress to file"""
    with open(PROGRESS_FILE, 'w') as f:
        json.dump({
            'last_row': last_row,
            'total_scanned': total_scanned,
            'updated': datetime.now().isoformat()
        }, f)

def backup_results():
    """Merge repo results into backup file"""
    if not os.path.exists(REPO_RESULTS):
        return
    
    # Load current results
    with open(REPO_RESULTS, 'r') as f:
        current = json.load(f)
    
    if not current:
        return
    
    # Load existing backup
    existing = []
    if os.path.exists(BACKUP_RESULTS):
        with open(BACKUP_RESULTS, 'r') as f:
            existing = json.load(f)
    
    # Merge (avoid duplicates)
    existing_subs = {r['subdomain'] for r in existing}
    new = [r for r in current if r['subdomain'] not in existing_subs]
    merged = existing + new
    
    # Save
    with open(BACKUP_RESULTS, 'w') as f:
        json.dump(merged, f, indent=2)
    
    return len(merged), len(new)

print("=" * 80)
print("SCAN WITH AUTO-RESUME")
print("=" * 80)
print("")

# Determine starting row
if FORCE_RESTART:
    print("🔄 FORCE_RESTART = True → Starting from row 1")
    if os.path.exists(PROGRESS_FILE):
        os.remove(PROGRESS_FILE)
    start_row = 1
    total_scanned = 0
else:
    progress = load_progress()
    start_row = progress['last_row'] + 1
    total_scanned = progress['total_scanned']
    if start_row > 1:
        print(f"📂 Resuming from saved progress!")
        print(f"   Previously scanned: {total_scanned:,} domains")
        print(f"   Last completed row: {progress['last_row']:,}")
        print(f"   → Starting from row: {start_row:,}")
    else:
        print("📝 No previous progress → Starting from row 1")
print("")

# Show backup status
if os.path.exists(BACKUP_RESULTS):
    with open(BACKUP_RESULTS, 'r') as f:
        backup_count = len(json.load(f))
    print(f"📦 Backup contains: {backup_count} results")
    print("")

# Check if already complete
if start_row > SCAN_END_ROW:
    print("✅ SCAN COMPLETE!")
    print(f"   All {SCAN_END_ROW:,} rows have been processed.")
    print(f"   Total scanned: {total_scanned:,}")
    print("")
    print("To restart, set FORCE_RESTART = True")
else:
    print(f"Source: df_base_filtered (sorted by {SORT_BY})")
    print(f"Total in dataset: {len(df_base_filtered):,} rows")
    print(f"Scan range: Row {start_row:,} to {SCAN_END_ROW:,}")
    print("")

    # Select remaining rows
    df_scan = df_base_filtered.iloc[start_row-1:SCAN_END_ROW].copy()
    domains_this_session = len(df_scan)
    print(f"Domains this session: {domains_this_session:,}")
    print("")

    # Create data directory and save targets
    data_dir = Path('/kaggle/working/subdomain-playground/data')
    data_dir.mkdir(parents=True, exist_ok=True)
    scans_dir = data_dir / 'scans'
    scans_dir.mkdir(exist_ok=True)
    
    targets_file = data_dir / 'all_sources.txt'
    df_scan['Subdomain'].to_csv(targets_file, index=False, header=False)
    print(f"✓ Saved to: {targets_file}")
    print("")

    # Setup environment
    os.chdir('/kaggle/working/subdomain-playground')
    os.environ['PATH'] = f"/kaggle/working/subdomain-playground/bin:{os.environ['PATH']}"
    os.environ['SUBFINDER_PATH'] = '/kaggle/working/subdomain-playground/bin/subfinder'
    os.environ['FINDOMAIN_PATH'] = '/kaggle/working/subdomain-playground/bin/findomain'
    os.environ['DNSX_PATH'] = '/kaggle/working/subdomain-playground/bin/dnsx'
    os.environ['HTTPX_PATH'] = '/kaggle/working/subdomain-playground/bin/httpx'
    os.environ['SUBZY_PATH'] = '/kaggle/working/subdomain-playground/bin/subzy'

    print("=" * 80)
    print("STARTING SCAN")
    print("=" * 80)
    print("")

    # Run scan
    process = subprocess.Popen(
        [sys.executable, '-u', 'scan.py',
         '-l', 'data/all_sources.txt',
         '--mode', 'full',
         '--require-cname-contains', 'shopify',
         '--filter-status', '3*,403,404,409,5*',
         '--workers', '2'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )

    # Track progress from output
    progress_pattern = re.compile(r'\[(\d+)/(\d+)\]')
    domains_completed = 0
    
    for line in process.stdout:
        print(line, end='', flush=True)
        
        # Parse [X/Y] progress
        match = progress_pattern.search(line)
        if match:
            domains_completed = int(match.group(1))
            
            # Save progress every 50 domains
            if domains_completed % 50 == 0:
                current_row = start_row + domains_completed - 1
                save_progress(current_row, total_scanned + domains_completed)
                # Also backup results periodically
                if domains_completed % 200 == 0:
                    backup_results()

    process.wait()
    
    # Final save
    final_row = start_row + domains_completed - 1 if domains_completed > 0 else start_row - 1
    final_total = total_scanned + domains_completed
    save_progress(final_row, final_total)
    
    # Backup results
    backup_info = backup_results()
    
    print("")
    print("=" * 80)
    print(f"Session complete!")
    print(f"   Domains this session: {domains_completed:,}")
    print(f"   Total scanned so far: {final_total:,}")
    print(f"   Last row completed: {final_row:,}")
    print(f"   Remaining: {SCAN_END_ROW - final_row:,} rows")
    if backup_info:
        print(f"   Results backed up: {backup_info[0]:,} total ({backup_info[1]} new)")
    print("")
    print("Progress and results saved. Re-run this cell to continue.")
    print("=" * 80)

## Cell 9: View Results

Views results from the backup file (`/kaggle/working/all_results.json`) which persists across sessions.

In [ ]:
import json
import os
from pathlib import Path

# Try backup file first, then repo file
BACKUP_RESULTS = '/kaggle/working/all_results.json'
REPO_RESULTS = '/kaggle/working/subdomain-playground/data/scans/shopify_takeover_candidates.json'

results = []
results_file = None

if os.path.exists(BACKUP_RESULTS):
    results_file = BACKUP_RESULTS
    with open(BACKUP_RESULTS, 'r') as f:
        results = json.load(f)
elif os.path.exists(REPO_RESULTS):
    results_file = REPO_RESULTS
    with open(REPO_RESULTS, 'r') as f:
        results = json.load(f)

if results:
    print("=" * 80)
    print("SCAN RESULTS")
    print("=" * 80)
    print("")
    print(f"Source: {results_file}")
    print(f"Total candidates found: {len(results)}")

    # Count by risk level
    risk_counts = {}
    for r in results:
        risk = r.get("risk_level", "unknown")
        risk_counts[risk] = risk_counts.get(risk, 0) + 1

    print("")
    print("Breakdown by risk:")
    for risk, count in sorted(risk_counts.items()):
        print(f"  {risk.upper()}: {count}")

    print("")
    print("=" * 80)
    print("TOP 20 FINDINGS")
    print("=" * 80)

    sorted_results = sorted(results, key=lambda x: x.get("confidence_score", 0), reverse=True)

    for i, r in enumerate(sorted_results[:20], 1):
        print("")
        print(f"{i}. {r['subdomain']}")
        print(f"   CNAME: {r.get('cname', 'N/A')}")
        print(f"   HTTP Status: {r.get('http_status', 'N/A')}")
        print(f"   Risk: {r.get('risk_level', 'N/A')}")
        print(f"   Confidence: {r.get('confidence_score', 0)}")
else:
    print("No results found yet.")
    print("Run Cell 9 (scan) first.")